<a href="https://colab.research.google.com/github/JapiKredi/Google_Gemma_LLM_Finetuning/blob/main/Google_Gemma_LLM_Finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gemma
The Gemma Model, a suite of lightweight open-source generative AI (GenAI) models created by Google DeepMind, targets developers and researchers. It was launched alongside Gemini, Google's proprietary generative AI chatbots.

Within the Gemma collection, two primary models stand out: Gemma 2B and Gemma 7B. These models are large language models (LLMs) designed for text-to-text decoding, each offering pretrained and instruction-tuned variations. Gemma 2B features a neural network with 2 billion parameters, while Gemma 7B boasts seven billion parameters.

Google provides pretrained and instruction-tuned Gemma models optimized for use on laptops and workstations, accessible to developers across multiple platforms. Additionally, Meta's Llama 2 serves as another open-source AI model tailored for laptop use, positioned more as a business tool compared to Gemma. Gemma is commonly preferred for scientific endeavors, while Llama 2 is seen as better suited for general-purpose tasks.

# Import Libraries

huggingface_hub: This library provides access to models, datasets, and other resources shared by the Hugging Face community.

transformers: Formerly known as pytorch-transformers or pytorch-pretrained-bert, this library is developed by Hugging Face. It provides state-of-the-art pre-trained models for natural language understanding (NLU) and natural language generation (NLG) tasks.

accelerate: Accelerate is a library developed by Hugging Face that simplifies distributed training for deep learning models and provides an easy-to-use interface for distributed computing frameworks.

BitsAndBytes: This library provides functions and utilities for working with binary data in Python. It includes functions for performing bitwise operations.

trl: The Text Representation Learning (TRL) library is developed by Hugging Face and provides tools and utilities for training and fine-tuning text representations.

peft: PEFT (PyTorch Extensible Fine-Tuning) is a library that extends PyTorch for fine-tuning large language models (LLMs) such as GPT and BERT.

In [ ]:
!pip install -q -U huggingface_hub
!pip install -q -U transformers
!pip install -q -U accelerate
!pip install -q -U BitsAndBytes
%pip install -q trl
%pip install -q peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 6.3 MB/s eta 0:00:00
